In [1]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
%matplotlib inline

### Network Architecture (DeepVO AlexNetLike)

In [2]:
from keras.layers import Input, Dense, Activation, Flatten, merge
from keras.models import Model
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam


tf.python.control_flow_ops = tf

# image_1 = Input(shape = (3, 256, 256), name='image1')
# image_2 = Input(shape = (3, 256, 256), name='image2')

# N_img_height = 256
# N_img_width = 256
# N_img_channels = 3

# inputShape = (N_img_height, N_img_width, N_img_channels)
def DeepVo():
    # TODO: Maybe add zeroPadding2D(1, 1) after each layer
    # TODO: Maybe take out input_shape = inputShape from first conv layer
    
    # Loading inputs: 
    image_1 = Input(shape = (256, 256, 3), name='image1')
    image_2 = Input(shape = (256, 256, 3), name='image2')
    
    ######## LAYER 1 ####################
    # Convolution Layer 1: Channel 1
    x1 = Convolution2D(48, 11, 11, subsample=(4, 4), 
                      init = 'he_normal', 
                      border_mode = 'valid', 
                      activation = 'relu',
#                       input_shape = inputShape,  # may not be necessary
                      name = 'conv_1_x')(image_1)
    
    # Convolution Layer 1: Channel 2
    y1 = Convolution2D(48, 11, 11, subsample=(4, 4), 
                      init = 'he_normal', 
                      border_mode = 'valid', 
                      activation = 'relu',
#                       input_shape = inputShape,  # may not be necessary
                      name = 'conv_1_y')(image_2)
    
    # Max Pooling Layer 1: Channel 1
    x1 = MaxPooling2D((3, 3), strides = (2, 2))(x1)
    
    # Max Pooling Layer 1: Channel 2
    y1 = MaxPooling2D((3, 3), strides = (2, 2))(y1)
    ######## END OF LAYER 1 ####################

    ######## LAYER 2####################
    # Convolution Layer 2: Channel 1
    x1 = Convolution2D(128, 5, 5, subsample = (1, 1), 
                      init = 'he_normal', 
                      border_mode = 'valid', 
                      activation = 'relu', 
                      name = 'conv_2_x')(x1)
    
    # Convolution Layer 2: Channel 2
    y1 = Convolution2D(128, 5, 5, subsample = (1, 1), 
                      init = 'he_normal', 
                      border_mode = 'valid', 
                      activation = 'relu', 
                      name = 'conv_2_y')(y1)
    
    # Max Pooling Layer 1: Channel 1
    x1 = MaxPooling2D((3, 3), strides = (2, 2))(x1)
    
    # Max Pooling Layer 1: Channel 2
    y1 = MaxPooling2D((3, 3), strides = (2, 2))(y1)
    
    ####### LAYER 3 ###################
    # Convolution Layer 3: Channel 1
    x1 = Convolution2D(192, 3, 3, subsample = (1, 1), 
                  init = 'he_normal', 
                  border_mode = 'valid', 
                  activation = 'relu', 
                  name = 'conv_3_x')(x1)

    # Convolution Layer 3: Channel 2
    y1 = Convolution2D(192, 3, 3, subsample = (1, 1), 
                  init = 'he_normal', 
                  border_mode = 'valid', 
                  activation = 'relu', 
                  name = 'conv_3_y')(y1)
    
    ####### LAYER 4 ###################
    # Convolution Layer 4: Channel 1
    x1 = Convolution2D(192, 3, 3, subsample = (1, 1), 
                  init = 'he_normal', 
                  border_mode = 'valid', 
                  activation = 'relu', 
                  name = 'conv_4_x')(x1)

    # Convolution Layer 4: Channel 2
    y1 = Convolution2D(192, 3, 3, subsample = (1, 1), 
                  init = 'he_normal', 
                  border_mode = 'valid', 
                  activation = 'relu', 
                  name = 'conv_4_y')(y1)
    
    ####### LAYER 5 ###################
    # Convolution Layer 5: Channel 1
    x1 = Convolution2D(192, 3, 3, subsample = (1, 1), 
                  init = 'he_normal', 
                  border_mode = 'valid', 
                  activation = 'relu', 
                  name = 'conv_5_x')(x1)

    # Convolution Layer 5: Channel 2
    y1 = Convolution2D(192, 3, 3, subsample = (1, 1), 
                  init = 'he_normal', 
                  border_mode = 'valid', 
                  activation = 'relu', 
                  name = 'conv_5_y')(y1)
    
    ####### Flatten ###########
    x2 = Flatten(name='flatten_x1')(x1)
    y2 = Flatten(name='flatten_y1')(y1)
    
    # TODO: Maybe add a relu after these layers
    ###### Fully Connected 1 using Xaviers algorithm (glorot normal) ########
    x3 = Dense(2048, activation = 'relu', init = 'glorot_normal', name = 'fc1_x2')(x2)
    y3 = Dense(2048, activation = 'relu', init = 'glorot_normal', name = 'fc1_y2')(y2)
    
    ###### Fully Connected 2 ########
    x3 = Dense(2048, activation = 'relu', init = 'glorot_normal', name = 'fc2_x2')(x2)
    y3 = Dense(2048, activation = 'relu', init = 'glorot_normal', name = 'fc2_y2')(y2)
    
    ##### Merge ####### 
    # TODO: Try concatenation instead of merging
    print('x3 shape: ', x3.get_shape())
    m3 = merge([x3, y3], mode='concat', concat_axis = 1, name = 'merge_x3y3')
    
    print('m3 shape: ', m3.get_shape())
    #### Flatten #####
#     flat3 = Flatten(name = 'flatten_merge')(m3)
    
    #### Fully connected 3 ######
    fc3 = Dense(4096, activation = 'relu', init = 'glorot_normal', name = 'fc3')(m3)
    
    ### Fully connected 4 ######
    fc4 = Dense(1024, activation = 'relu', init = 'glorot_normal', name = 'fc4')(fc3)
    
    ### Fully connected 5 ######
    prediction = Dense(1, name = 'output')(fc4)
    
    model = Model(input = [image_1, image_2], output = [prediction, prediction])
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer = adam, loss = 'mse')
    
    return model
    
    

Using TensorFlow backend.


### Manually create testing data

### Train Generator

In [11]:
X_train = np.asarray([img_1, img_2])

In [12]:
img1 = np.asarray(img_1, dtype=np.uint8)
img2 = np.asarray(img_2, dtype=np.uint8)
X_train = np.asarray([img1, img2])

In [13]:
y_train = [[14.136534929275513, 5.7756648244489179], [14.195453882217407, 5.751016049309901]]

In [18]:
def generate_training_data(image_data, label):
    # image_data array with two images
    # label is the ground truth tuple [time, speed]
    img1 = image_data[0]
    img2 = image_data[1]
    s1 = label[0][1]
    s2 = label[1][1]
    ds = abs(s2 - s1)
    print('img1: ', img1.shape)
    print('img2: ', img2.shape)
    print('ds: ', ds)
    
    
    yield (img1, img2, ds)

#### Create the model

In [19]:
model = DeepVo()



x3 shape:  (?, 2048)
m3 shape:  (?, 4096)


In [20]:
BATCH = 2

In [21]:
train_generator = generate_training_data(X_train, y_train)
history = model.fit_generator(train_generator, samples_per_epoch = 1, nb_epoch = 1)

img1: Epoch 1/1 (204, 204, 3)

img2:  (256, 256, 3)
ds:  0.024648775139016976


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/jj/anaconda2/envs/python3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/jj/anaconda2/envs/python3/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jj/anaconda2/envs/python3/lib/python3.5/site-packages/keras/engine/training.py", line 425, in data_generator_task
    generator_output = next(generator)
StopIteration



Exception: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None

In [ ]:
modelx = DeepVo()
print('model created!')
